<a href="https://colab.research.google.com/github/johnsl01/income/blob/master/incomereg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#_
_Multiple variable (hyperplane) regression_
Income data 


##_
_ml outline approach_

A structured ML student exercise type task typically comprises a set of data with one or more answers per data sample and a second set of data with the answers missing.

The basic task is to use machine learning to provide the missing answers.

The normal approach is : 

Explore and understand the data provided - typically producing some basic metrics around the data and preparing some simple correlations between elements of the data and the answers.

Assess the quality of the data - normally carried out as part of the exploration this task is more concentrating on the missing and bad data and any extreme outliers - and also looking at any unstructured data (free text strings etc) and determining what remediation is necessary to either impute the missing data - discard samples or manipulate the unstructured data to generate structured elements from it.

Carry out some initial ML tasks such as linear (planar) regression and assess the quality of the results on the known answers and provide an initial version of the answers to use as a benchmark for more sophisticated next steps.

Make an assesssment based on the information gathered above on the approach to ML - what types of ML tools are approriate to the data and estimate what level of quality could be achieved. For a student / training exercise the requirement may be to consider more than one aproach and to compare the results - or the requirement may be simply to produce the best result, and in this case there may be a submission so the 'real' answers are hidden and only a score available to compare results.

Student exercise can also contain more specific requirements - to use a specific type of ML approach and to demonstrate the impacts of some parameter changes or to examine the impact of onder or over fitting to the test data.


##_
_discription and general notes_

y = B0 + B1.X1 + B2.X2 + B3.X3 ...

##_
imports and data loads

In [0]:
###############################################
#@title                 Imports               #
###############################################
print ("Imports")

import numpy as np

import pandas as pd

import sklearn

# !pip install fancyimpute
# import fancyimpute as fi

import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D

from google.colab import files

from datetime import datetime
print (datetime.now())



In [0]:
###############################################
#@title               Data loads              #
###############################################
print ("Data loads")
sourcedata = "https://raw.githubusercontent.com/johnsl01/income/master/incomeknown.csv"
incomeknown_df = pd.read_csv(sourcedata, sep=",")

print (datetime.now())

In [0]:
# look at the data 
incomeknown_df.head(10)

In [0]:
# dont like the col names - much easier with simple strings with no spaces.
incomeknown_df.rename(columns = {"Year of Record" : "Year",
                              "Size of City" : "CitySize",
                              "University Degree" : "Degree",
                              "Wears Glasses" : "Glasses",
                              "Hair Color" : "Hair",
                              "Body Height [cm]" : "Height",
                              "Income in EUR" : "Income"},
                      inplace = True)

print (datetime.now())

In [0]:
# look again 
incomeknown_df.tail(10)

##_
_Initial data exploration_

In [0]:
incomeknown_df.describe()



In [0]:
incomeknown_df[['Gender','Country','Profession','Degree','Hair']].describe()

In [0]:
print ( incomeknown_df['Profession'].value_counts() )

print ( incomeknown_df['Degree'].value_counts() )

print ( incomeknown_df['Hair'].value_counts() )

print ( incomeknown_df['Country'].value_counts() )

print ( incomeknown_df['Gender'].value_counts() )



In [0]:
# prepare some simple histograms and scatter plots of numeric data against income
# looking for any obvious correlation

# exclude outliers (particulalry high incomes) to make the plots more useful

# 

# incomeknown_df.hist("Year", bins=40)
# incomeknown_df.loc[incomeknown_df.Income < 1000000].plot.scatter(x = "Year", y = "Income")

# print ("Scater plot of Income by Year")
plt.scatter(incomeknown_df.loc[incomeknown_df.Income < 60000]["Year"], incomeknown_df.loc[incomeknown_df.Income < 60000]["Income"], alpha=0.0075)
plt.title('Scatter plot of Income by Year')

incomeknown_df.loc[incomeknown_df.Income < 500000].hist("Income", bins=300)


# incomeknown_df.hist("Age")
# incomeknown_df.loc[incomeknown_df.Income < 1000000].plot.scatter(x = "Age", y = "Income")

incomeknown_df.hist("Height", bins=200)
incomeknown_df.loc[incomeknown_df.Income < 100000].plot.scatter(x = "Height", y = "Income", alpha=0.005)


incomeknown_df.loc[(incomeknown_df.Income < 100000) & 
                  (incomeknown_df.Gender == "male")].hist("Height", bins=300)

incomeknown_df.loc[(incomeknown_df.Income < 100000) & 
                   (incomeknown_df.Gender == "male")].plot.scatter(x = "Height", y = "Income", c="LightBlue", alpha=0.01)

incomeknown_df.loc[(incomeknown_df.Income < 100000) & 
                   (incomeknown_df.Gender == "female")].hist("Height", bins=300)

incomeknown_df.loc[(incomeknown_df.Income < 100000) & 
                   (incomeknown_df.Gender == "female")].plot.scatter(x = "Height", y = "Income", c = 'Pink', alpha=0.01)

incomeknown_df.loc[(incomeknown_df.Income < 100000) & 
                   ((incomeknown_df.Gender != "female") & (incomeknown_df.Gender != "male"))].hist("Height", bins=300)

incomeknown_df.loc[(incomeknown_df.Income < 100000) & 
                   ((incomeknown_df.Gender != "female") & (incomeknown_df.Gender != "male"))].plot.scatter(x = "Height", y = "Income", c = 'Grey', alpha=0.02)


In [0]:
incomeknown_df.loc[incomeknown_df.Age > 105]
# incomeknown_df.loc[incomeknown_df.Income > 2000000]

##_
function definitions

In [0]:
print("def predict(X, theta)")
def predict(X, theta):
  # takes m by n matrix X as input and returns an m by 1 vector 
  # containing the predictions h_theta(x^i) for each row x^i, i=1,...,m in X
  ##### replace the next line with your code #####
  
  # This is the generic multiple variable implementation. 
  
  # For conveniece each data point in X has an initial 1 
  #   added to enable the B0 coeficient to be applied efficiently
  
  # print(X)
  # print(theta)
  # print(X.shape)
  
  # Get the number of coeficients B0, B1 ..... 
  #   we iterate over these
  n=len(theta)
  
  # For the multiple linear regression we need to iterate over the 
  #   length of theta 
  # This assumes a simple B0 + B1.X1 + B2.X2 + ....
  #   it does not consider more complex dependencies
  #   such as B0 + B1.X1 + B2.X2 + B3.X1.X2   etc.
  #   these require the X to be constructed to meet the requirement
  #   and may introduce too much colinearity.
  
  # Check for single data point
  #   requires a special case as it arrives as 1D, rather than 2D
  #   and if it does arrive as 2D the else handles it anyway
  
  # Numpy doesn't appear to support a mechanism to directly 
  #   multiply each row of an array by an equivalent sized vector
  #   so we use a for loop to iterate over the vector but use 
  #   numpy maths, which are more efficient, to apply each element
  #   of the theta vetor to the equivalent data variable for all points. 
   
  if len(X.shape) == 1 : 
    # this is the edge case for a single data point in a vector
    pred = X[0] * theta[0] 
    for i in range(1,n) :
      pred = pred + X[i]*theta[i]  
        
  else : 
    #this is the general case for data point(s) in an array
    pred = X[:,0]*theta[0] 
    for i in range(1,n) :
      pred = pred + X[:,i]*theta[i]    
        
  # print(pred)
  
  # Return the prediction
  # The returned value is a vector of value for each data point.
  return pred
# end def predict


print("def computeCost(X, y, theta)")
def computeCost(X, y, theta):
  # function calculates the cost J(theta) and return its value
  ##### replace the next line with your code #####
  
  # this function is already independent of the number of variables
  # provided the X array carries the data with a leading 1 on each data point
  # and the theta has the same number of B0, B1, B2 ...  values.
  
  # Get the value(s) predicted by the current theta values
  costpred = predict (X,theta)
  
  # get the difference(s) between the predictions and the actuals
  costbase = costpred - y
    
  # get the square of the difference(s)
  costsq = costbase **2
  
  # get the sum of the squared difference(s)
  sumcost = costsq.sum() 
  
  # divide the sum of squares by twice the number of data points
  cost = sumcost/(2*len(y))  
  
  # return the cost of the current theta values
  # the returned value is a numeric value of the cost.
  return cost
# end def computeCost



print("def computeGradient(X, y, theta)")
def computeGradient(X, y, theta):
  # function calulate the gradient of J(theta) and returns its value
  ##### replace the next line with your code #####
  # This function is already capable of delaing with multiple varables
  # provided X (with a leading 1 per data point) and theta are matched sizes.
  
  # number of coeficients
  n=len(theta)
  # initiate the result
  grad = np.zeros(n)
  
  # print ("Number of coeficients: " , n)
  # print ("Shapes of data, result and theta : " , X.shape, y.shape, theta.shape)
  # print ("Types of  data, result and theta : " , type(X), type(y), type(theta))
  # print ("Sample head of data")
  # for i in range(5) :
      # print (X[i,:])
    
  # Get the value predicted by the current theta values
  costpred = predict (X,theta)
  # print ("Compute Grad #2 :", costpred.shape)
  
  # get the difference between the predictions and the actuals
  costbase = costpred - y
  # print ("Compute Grad #3 :", costbase.shape)  
  
  # calculate the gradient for each coefficient
  # for is inefficient but it is used to iterate over a small
  # range (1 more than the number of variables 
  # i.e the number of coefficients)
  # while numpy array maths are used for the larger dimension
  # (the number of data points)
  for i in range(n) :
    # get product of differences by current coeficients data point
    costprod = costbase * X[:,i]
    # print ("Compute Grad #4 :", costprod.shape) 
    # get the sum of the cost products
    costprodsum = costprod.sum()
    # print ("Compute Grad #5 :", costprodsum)
    # divide by the number of data points
    grad[i] = costprodsum / len(y)
    # take this outside the for loop and use numpy maths ! 
    # print ("Compute Grad #6 :", grad[i])
  # end for
  # print ("Compute Grad #6 :", grad)
  
  # return the gradient
  # the returned value of a vector of a gradient for each coeficient (theta) 
  return grad
# end def computeGradient


print("def gradDescent(X, y, theta, iters, alpha)")
def gradDescent(X, y, theta, iters, alpha):
  # X - data (with the extra 1s in col 1)
  # y - results - vector with 1 resilt per data point
  # theta - B0, B1, B2 ... starting values 
  # iters - how many iterations
  # alpha - adjustor for size of adjustment per step
  
  # returns : 
  # theta - B0, B1, B2 ... final values 
  # cost - cost after each iteration
  
  # initialize cost array
  cost = np.zeros(iters)
  
  for i in range(iters):
    theta = theta - alpha * computeGradient(X,y,theta)
    cost[i] = computeCost(X, y, theta)

  return theta, cost
# end def gradientDescent   
  

print (datetime.now())

In [0]:

print("def gradientDescent(X, y, numparams)")
def gradientDescent(X, y, numparams):
  # iteratively update parameter vector theta
  # -- you should not modify this function

  # initialize variables for learning rate and iterations
  alpha = 0.02
  iters = 5000
  cost = np.zeros(iters)
  theta= np.zeros(numparams)

  for i in range(iters):
    theta = theta - alpha * computeGradient(X,y,theta)
    cost[i] = computeCost(X, y, theta)

  return theta, cost



print("def normaliseData(x)")
def normaliseData(x):
  # rescale data to lie between 0 and 1
  scale = x.max(axis=0)
  return (x/scale, scale)



print("def splitData(X, y)")
def splitData(X, y):
  # split data into training and test parts
  # ... for now, we use all of the data for training and testing
  Xtrain=X; ytrain=y; Xtest=X; ytest=y
  return (Xtrain, ytrain, Xtest, ytest)


print (datetime.now())

##_
_main_

In [0]:
print ("def main()")
def main():
  # load the data
  # "https://raw.githubusercontent.com/johnsl01/Titanic_Python/master/titanic_known.csv"
  data=np.loadtxt('https://raw.githubusercontent.com/johnsl01/linreg/master/stockprices.csv',usecols=(1,2))
  X=data[:,0]
  y=data[:,1]

  # plot the data so we can see how it looks
  # (output is in file graph.png)
  fig, ax = plt.subplots(figsize=(12, 8))
  ax.scatter(X, y, label='Data')
  ax.set_xlabel('Amazon')
  ax.set_ylabel('Google')
  ax.set_title('Google stock price vs Amazon')
  fig.savefig('graph.png')

  # split the data into training and test parts
  (Xtrain, ytrain, Xtest, ytest)=splitData(X,y)

  # add a column of ones to input data
  m=len(y) # m is number of training data points
  Xtrain = np.column_stack((np.ones((m, 1)), Xtrain))
  (m,n)=Xtrain.shape # m is number of data points, n number of features

  # rescale training data to lie between 0 and 1
  (Xt,Xscale) = normaliseData(Xtrain)
  (yt,yscale) = normaliseData(ytrain)

  # calculate the prediction
  print('testing the prediction function ...')
  theta=(1,2)
  print('when x=[1,1] and theta is [1,2]) cost = ',predict(np.ones(n),theta))
  print('approx expected prediction is 3')
  print('when x=[[1,1],[5,5]] and theta is [1,2]) cost = ',predict(np.array([[1,1],[5,5]]),theta))
  print('approx expected prediction is [3,15]')
  input('Press Enter to continue...')

  # calculate the cost when theta iz zero
  print('testing the cost function ...')
  theta=np.zeros(n)
  print('when theta is zero cost = ',computeCost(Xt,yt,theta))
  print('approx expected cost value is 0.318')
  input('Press Enter to continue...')

  # calculate the gradient when theta is zero
  print('testing the gradient function ...')
  print('when theta is zero gradient = ',computeGradient(Xt,yt,theta))
  print('approx expected gradient value is [-0.79,-0.59]')
  input('Press Enter to continue...')

  # perform gradient descent to "fit" the model parameters
  print('running gradient descent ...')
  theta, cost = gradientDescent(Xt, yt, n)
  print('after running gradientDescent() theta=',theta)
  print('approx expected value is [0.34, 0.61]')

  # plot some predictions
  Xpred = np.linspace(X.min(), X.max(), 100)
  Xpred = np.column_stack((np.ones((100, 1)), Xpred))
  ypred = predict(Xpred/Xscale, theta)*yscale
  fig, ax = plt.subplots(figsize=(12, 8))
  ax.scatter(Xtest, ytest, color='b', label='Test Data')
  ax.plot(Xpred[:,1], ypred, 'r', label='Prediction')
  ax.set_xlabel('Amazon')
  ax.set_ylabel('Google')
  ax.legend(loc=2)
  fig.savefig('pred.png')

  # and plot how the cost varies as the gradient descent proceeds
  fig2, ax2 = plt.subplots(figsize=(12, 8))
  ax2.semilogy(cost,'r')
  ax2.set_xlabel('iteration')
  ax2.set_ylabel('cost')
  fig2.savefig('cost.png')
  
  # plot the cost function
  fig3 = plt.figure()
  ax3 = fig3.add_subplot(1, 1, 1, projection='3d')
  n=100
  theta0, theta1 = np.meshgrid(np.linspace(-3, 3, n), np.linspace(-3, 2, n))
  cost = np.empty((n,n))
  for i in range(n):
    for j in range(n):
      cost[i,j] = computeCost(Xt,yt,(theta0[i,j],theta1[i,j]))
  ax3.plot_surface(theta0,theta1,cost)
  ax3.set_xlabel('theta0')
  ax3.set_ylabel('theta1')
  ax3.set_zlabel('J(theta)')
  fig3.savefig('J.png')
  
# end def main()  
  
print (datetime.now())


In [0]:
print (datetime.now())

main()

print (datetime.now())


##_ 
_testing section_

In [0]:
# test section (predict)
# assume all defs are in place 
# but main hasn't run
print("Test of multi variable prediction")
print("Three variables - five data points - with B0")
test_array = np.array([[1,2,3,5],
                       [1,7,11,13],
                       [1,17,19,23],
                       [1,29,31,37],
                       [1,41,43,47]])
print ("shape of test data : " , test_array.shape )
test_theta = np.array([1,2,3,5])
print ("shape of test theta : ", test_theta.shape )

test_predict = predict (test_array, test_theta)
print(test_predict)



In [0]:
print ("Test of multi variable cost")
print ("cost when theta is all zeros and when theta is correct")
test_array = np.array([[1,2,3,5],
                       [1,7,11,13],
                       [1,17,19,23],
                       [1,29,31,37],
                       [1,41,43,47]])
print ("shape of test data : " , test_array.shape )
test_theta = np.array([0,0,0,0])
print ("shape of test theta : ", test_theta.shape )
test_results = np.array([22,61,117,191,261])
print ("shape of test results : ", test_results.shape )
good_theta = np.array([5,3,2,1])
print ("shape of correct theta : ", good_theta.shape )

test_cost =  computeCost(test_array, test_results, test_theta)
print ("cost at theta all zeros : " , test_cost)

good_cost =  computeCost(test_array, test_results, good_theta)
print ("cost at correct theta : " , good_cost)

In [0]:
print ("Test of Gradient Descent")
print ("At zero theta and correct theta")
test_array = np.array([[1,2,3,5],
                       [1,7,11,13],
                       [1,17,19,23],
                       [1,29,31,37],
                       [1,41,43,47]])
print ("shape of test data : " , test_array.shape )
test_theta = np.array([0,0,0,0])
print ("shape of test theta : ", test_theta.shape )
test_results = np.array([22,61,117,191,261])
print ("shape of test results : ", test_results.shape )
good_theta = np.array([5,3,2,1])

test_grad =  computeGradient(test_array, test_results, test_theta)
print ("gradient at theta all zeros : " , test_grad)

good_grad =  computeGradient(test_array, test_results, good_theta)
print ("gradient at theta correct : " , good_grad)

In [0]:
# full implementation with multi variable test data 
X = np.array([[2,3,5],
              [7,11,13],
              [17,19,23],
              [29,31,37],
              [41,43,47]])
print ("shape of test data : " , X.shape )
print ("test data : \n ", X)

# perfectly linear results
# y = np.array([22,61,117,191,261])

# Or with some some non linearity 
y = np.array([23,60,118,193,260])

print ("shape of test results : ", y.shape )
print ("results : " , y )

good_theta = np.array([5,3,2,1])

# Starting Position 
# note : without col of 1s in data (they get added below)


# split the data into training and test parts
#   note : current split is 100% train
(Xtrain, ytrain, Xtest, ytest)=splitData(X,y)

# add a column of ones to input data
m=len(ytrain) # m is number of training data points
# note this is a flaw in the original code as it was len(y) which only works if
# the train/test split is 100% - so it needs to be len (ytrain) to avoid confusion
Xtrain = np.column_stack((np.ones((m)), Xtrain))

print ("Shape of training data : " , Xtrain.shape)
print ("Training Data : \n " , Xtrain)
 
(m,n)=Xtrain.shape # m is number of data points, n number of features

# rescale training data to lie between 0 and 1
(Xt,Xscale) = normaliseData(Xtrain)
(yt,yscale) = normaliseData(ytrain)

print ("Shape of training data : " , Xt.shape)
print ("Training Data : \n " , Xt)

# perform gradient descent to "fit" the model parameters
# print('running gradient descent ...')
# theta, cost = gradientDescent(Xt, yt, n)
# print('after running gradientDescent() theta=',theta , "\n at cost : " , cost)

theta_init = np.zeros(n)
iters = 1000000
alpha = 0.1

initial_cost = computeCost ( Xt, yt, theta_init)
print ("Initial theta : " , theta_init )
print ("Initial cost = ", initial_cost)


print('running gradient descent ...')
theta_new, cost = gradDescent(Xt, yt, theta_init, iters, alpha)
# print('after running gradientDescent() theta=',theta , "\n at cost : " , cost)

final_cost = cost[len(cost)-1]

print ("final theta : " , theta_new )
print ("final cost = ", final_cost)


# and plot how the cost varies as the gradient descent proceeds
fig2, ax2 = plt.subplots(figsize=(12, 8))
ax2.semilogy(cost,'r')
ax2.set_xlabel('iteration')
ax2.set_ylabel('cost')
# fig2.savefig('cost.png')

theta_init = theta_new
iters = 5
alpha = 0.1

initial_cost = computeCost ( Xt, yt, theta_init)
print ("Initial theta : " , theta_init )
print ("Initial cost = ", initial_cost)


print('running gradient descent ...')
theta_new, cost = gradDescent(Xt, yt, theta_init, iters, alpha)
# print('after running gradientDescent() theta=',theta , "\n at cost : " , cost)

final_cost = cost[len(cost)-1]

print ("final theta : " , theta_new )
print ("final cost = ", final_cost)


# and plot how the cost varies as the gradient descent proceeds
fig3, ax3 = plt.subplots(figsize=(20, 8))
ax3.semilogy(cost,'r')
ax3.set_xlabel('iteration')
ax3.set_ylabel('cost')
# fig2.savefig('cost.png')

In [0]:
print (test_array, "\n")
print (test_array * 2, "\n")

In [0]:
print (test_array + test_array, "\n")
print (test_array[0,:], "\n")
print (test_array[1,:], "\n")
print (test_array[2,:], "\n")
print (test_array[:,0], "\n")
print (test_array[:,1], "\n")

In [0]:
predict_1 = np.zeros((test_array.shape[0],test_array.shape[1]))
print (predict_1.shape, "\n")
print (predict_1, "\n")

print (predict_1[0,:], "\n")
print (predict_1[1,:], "\n")
print (predict_1[2,:], "\n")
print (predict_1[:,0], "\n")
print (predict_1[:,1], "\n")

In [0]:
testnum = "Case #1"
myX = np.array([1,1])
mytheta = (1,2)

In [0]:
testnum = "Case #2"
myX = np.array([[1,1],[5,5]])
mytheta = (1,2)

In [0]:
print (testnum)
print (datetime.now(), "\n")
print ("Shape of myX : ", myX.shape, "\n")
print ("Type of myX.shape : " , type (myX.shape), "\n")
print ("Len of myX.shape : ", len(myX.shape), "\n")
if len(myX.shape) == 1 : 
  print ("myX col 0 : ", myX[0], "\n")
  print ("myX col 1 : ", myX[1], "\n")
else : 
  print ("myX col 0 : ", myX[:,0], "\n")
  print ("myX col 1 : ", myX[:,1], "\n")
print (mytheta[0], "\n")
print (mytheta[1], "\n", "\n")
mypredict_1 = predict(myX,mytheta)
print (mypredict_1.shape, "\n")
print (mypredict_1, "\n", "\n")
print (datetime.now(), "\n")

In [0]:
# multi dimensional test example
# need two dependant variables 
# will produce a planar regression.
# need some data
# use the existing amazon and google data 
# but generate a new result variable
# based on an integer multiple of each of the variables 
# with a randon factor thrown into each plus an additional 
# randon factor.
data=np.loadtxt('https://raw.githubusercontent.com/johnsl01/linreg/master/stockprices.csv',usecols=(1,2))
# X=data[:,0]
# y=data[:,1]
X = data
y = X[:,0]*3 
y = y + X[:,1]*2
print (X.shape, y.shape)





